In [27]:
import regrex as re


In [ ]:
import pandas as pd
import numpy as np
import random

def ensuciar_df(
    df_original,
    porcentaje_duplicados=0.3,
    porcentaje_invalid=0.02,
    porcentaje_nan=0.03,
    max_nan_col=10,
    max_intentos=30,
    verbose=True
):
    """Ensucia un DataFrame introduciendo duplicados, NaNs, errores de tipo y valores inválidos."""
    df = df_original.copy()

    for intento in range(max_intentos):
        df_temp = df.copy()

        # ----- 1. Duplicar filas -----
        num_duplicados = random.randint(1, int(len(df_temp) * porcentaje_duplicados))
        duplicadas = df_temp.sample(n=num_duplicados, replace=True, random_state=random.randint(0, 9999))
        df_temp = pd.concat([df_temp, duplicadas], ignore_index=True)

        # ----- 2. Insertar valores inválidos -----
        num_invalid = int(len(df_temp) * porcentaje_invalid)
        columnas_invalid = random.sample(df_temp.columns.tolist(), max(1, len(df_temp.columns) // 2))
        for col in columnas_invalid:
            if df_temp[col].dtype != "object":
                df_temp[col] = df_temp[col].astype("object")

            valid_indices = df_temp.index[df_temp[col].notna()]
            if len(valid_indices) > 0:
                invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
                df_temp.loc[invalid_indices, col] = "Auto%#"

        # ----- 3. Cambiar tipos de columnas -----
        columnas_a_cambiar = random.sample(df_temp.columns.tolist(), max(1, len(df_temp.columns) // 2))
        for col in columnas_a_cambiar:
            if pd.api.types.is_numeric_dtype(df_temp[col]):
                df_temp[col] = df_temp[col].astype(str)
            else:
                df_temp[col] = pd.to_numeric(df_temp[col], errors="coerce")

        # ----- 4. Insertar NaNs -----
        num_nan = int(len(df_temp) * porcentaje_nan)
        for column in df_temp.columns:
            nan_indices = np.random.choice(df_temp.index, size=min(num_nan, len(df_temp)), replace=False)
            df_temp.loc[nan_indices, column] = np.nan

        # ----- 5. Validar límite de NaNs -----
        nan_porcentaje = df_temp.isnull().mean() * 100
        if (nan_porcentaje > max_nan_col).any():
            if verbose:
                print(f"Intento {intento + 1}: Columna con más del {max_nan_col}% de NaNs. Reintentando...")
            continue
        else:
            if verbose:
                print(f"✅ DataFrame ensuciado correctamente en el intento {intento + 1}.")
            return df_temp

    raise ValueError(f"No se logró generar un DataFrame válido en {max_intentos} intentos.")

#Cargar csv y lerlo
ruta_csv = 'https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/Proyecto-cienciasdedatos/refs/heads/main/games%20steam%202013-2023.csv?token=GHSAT0AAAAAADMMHO5CLYHPHXXPCII4ELRK2HVAHQA'
df = pd.read_csv(ruta_csv)

df_sucio = ensuciar_df(df)

# Guardar el resultado
df_sucio.to_csv("df_sucio.csv", index=False)

# Reporte final
print("\n--- Reporte de NaNs ---")
print(df_sucio.isnull().sum())
print(f"\nTamaño final del DataFrame: {df_sucio.shape}")


Intento 1: Columna con más del 10% de NaNs. Reintentando...
Intento 2: Columna con más del 10% de NaNs. Reintentando...
Intento 3: Columna con más del 10% de NaNs. Reintentando...
Intento 4: Columna con más del 10% de NaNs. Reintentando...
Intento 5: Columna con más del 10% de NaNs. Reintentando...
Intento 6: Columna con más del 10% de NaNs. Reintentando...
Intento 7: Columna con más del 10% de NaNs. Reintentando...
Intento 8: Columna con más del 10% de NaNs. Reintentando...
Intento 9: Columna con más del 10% de NaNs. Reintentando...
✅ DataFrame ensuciado correctamente en el intento 10.

--- Reporte de NaNs ---
name            1867
release_date    1867
price           3079
positive        1867
negative        1867
app_id          1867
min_owners      1867
max_owners      1867
hltb_single     1867
dtype: int64

Tamaño final del DataFrame: (62257, 9)


In [ ]:
import pandas as pd
import numpy as np
import random

def ensuciar_df(
    df_original,
    columnas_protegidas=None,
    porcentaje_duplicados=0.3,
    porcentaje_invalid=0.02,
    porcentaje_nan=0.05,
    valor_invalid='bbb'
):
    """Ensucia un DataFrame introduciendo duplicados, valores inválidos, cambios de tipo y NaNs,
    excluyendo las columnas protegidas."""
    
    df = df_original.copy()
    columnas_protegidas = columnas_protegidas or []
    columnas_modificables = [c for c in df.columns if c not in columnas_protegidas]

    # 1️⃣ Duplicar algunas filas aleatorias (esto afecta todas las columnas, pero no modifica su contenido)
    num_duplicados = random.randint(1, int(len(df) * porcentaje_duplicados))
    duplicadas = df.sample(n=num_duplicados, replace=True, random_state=random.randint(0, 9999))
    df = pd.concat([df, duplicadas], ignore_index=True)

    # 2️⃣ Insertar valores inválidos en columnas aleatorias (excepto las protegidas)
    num_invalid = int(len(df) * porcentaje_invalid)
    columnas_invalid = random.sample(columnas_modificables, max(1, len(columnas_modificables) // 2))
    for col in columnas_invalid:
        if df[col].dtype != "object":
            df[col] = df[col].astype("object")

        valid_indices = df.index[df[col].notna()]
        if len(valid_indices) > 0:
            invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
            df.loc[invalid_indices, col] = valor_invalid

    # 3️⃣ Cambiar tipos de formato en columnas aleatorias (excepto las protegidas)
    columnas_a_cambiar = random.sample(columnas_modificables, max(1, len(columnas_modificables) // 2))
    for col in columnas_a_cambiar:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(str)
        else:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 4️⃣ Insertar NaNs en columnas modificables solamente
    num_nan = int(len(df) * porcentaje_nan)
    for column in columnas_modificables:
        nan_indices = np.random.choice(df.index, size=min(num_nan, len(df)), replace=False)
        df.loc[nan_indices, column] = np.nan

    return df

# Cargar el CSV y aplicar las modificaciones
ruta_csv = "https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/Proyecto-cienciasdedatos/refs/heads/main/games%20steam%202013-2023.csv?token=GHSAT0AAAAAADMMHO5CNDIYW5QPKFKZYRWU2HVAJVA"
df = pd.read_csv(ruta_csv)

# Columnas que no deben ensuciarse
columnas_protegidas = ["name", "release_date"]

# Intentos
max_intentos = 100

for intento in range(1, max_intentos + 1):
    df_sucio = ensuciar_df(df, columnas_protegidas=columnas_protegidas)

    # Calcular porcentaje de NaNs en columnas modificables
    nan_porcentaje = df_sucio.drop(columns=columnas_protegidas, errors="ignore").isnull().mean() * 100

    if (nan_porcentaje > 10).any():
        print(f"Intento {intento}: hay columnas con más del 10% de NaNs, reintentando...")
    else:
        print(f"✅ Finalización exitosa en el intento {intento}: no hay columnas con más del 10% de NaNs.")
        break
else:
    print("⚠️ Se alcanzó el número máximo de intentos sin lograr un DataFrame válido.")

# Guardar el DataFrame sucio
df_sucio.to_csv("df_sucio.csv", index=False)

# Mostrar resumen
print("\n--- Reporte final de NaNs ---")
print(df_sucio.isnull().sum())
print(f"\nTamaño final del DataFrame: {df_sucio.shape}")



Intento 1: hay columnas con más del 10% de NaNs, reintentando...
Intento 2: hay columnas con más del 10% de NaNs, reintentando...
✅ Finalización exitosa en el intento 3: no hay columnas con más del 10% de NaNs.

--- Reporte final de NaNs ---
name               0
release_date       0
price           3881
positive        3881
negative        5367
app_id          3881
min_owners      3881
max_owners      3881
hltb_single     3881
dtype: int64

Tamaño final del DataFrame: (77628, 9)


In [42]:
df_sucio.head()

,nombre,fecha_de_lanzamiento,precio,calificacion_positivas,calificacion_negativas,id_del_juego,minimo_de_dueños,maximo_de_dueños,duracion_de_la_campaña_aprox
0,Train Bandit,"Oct 12, 2017",0.99,53.0,5.0,655370,0,20000.0,NaN
1,Henosis™,"Jul 23, 2020",5.99,3.0,0.0,1355720,0,20000.0,NaN
2,Two Weeks in Painland,"Feb 3, 2020",0.00,50.0,8.0,1139950,0,NaN,NaN
3,Wartune Reborn,"Feb 26, 2021",0.00,87.0,49.0,NaN,50000,100000.0,NaN
4,TD Worlds,"Jan 9, 2022",10.99,21.0,7.0,1659180,0,20000.0,NaN


## **Inicio de la limpieza de la base de datos**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60952 entries, 0 to 60951
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          60952 non-null  object 
 1   release_date  60952 non-null  object 
 2   price         60952 non-null  float64
 3   positive      60952 non-null  int64  
 4   negative      60952 non-null  int64  
 5   app_id        60952 non-null  int64  
 6   min_owners    60952 non-null  int64  
 7   max_owners    60952 non-null  int64  
 8   hltb_single   12972 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.2+ MB


In [38]:
df.shape

(60952, 9)

In [ ]:
df_sucio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67420 entries, 0 to 67419
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          67420 non-null  object 
 1   release_date  67420 non-null  object 
 2   price         64049 non-null  float64
 3   positive      64049 non-null  float64
 4   negative      62759 non-null  float64
 5   app_id        64049 non-null  float64
 6   min_owners    62762 non-null  float64
 7   max_owners    64049 non-null  object 
 8   hltb_single   13616 non-null  float64
dtypes: float64(6), object(3)
memory usage: 4.6+ MB


In [ ]:
df_sucio.isnull().sum()

name                0
release_date        0
price            3371
positive         3371
negative         4661
app_id           3371
min_owners       4658
max_owners       3371
hltb_single     53804
dtype: int64

In [39]:
df_sucio.shape

(77628, 9)

In [50]:
import pandas as pd
import numpy as np

def limpiar_df(df, palabras_basura=None):
    """
    Limpia un DataFrame reemplazando NaNs por 0 y eliminando palabras basura.
    """
    df_limpio = df.copy()

    # 1️⃣ Reemplazar NaN con 0
    df_limpio = df_limpio.fillna(0)

    # 2️⃣ Definir palabras basura a eliminar
    if palabras_basura is None:
        palabras_basura = ["bbb", "Auto%#", "invalid", "error", "nan", "NaN"]

    # 3️⃣ Reemplazar palabras basura por 0 (sin afectar valores numéricos)
    for col in df_limpio.columns:
        if df_limpio[col].dtype == "object":
            df_limpio[col] = df_limpio[col].replace(palabras_basura, 0)
            # También eliminar espacios extra o caracteres raros
            df_limpio[col] = df_limpio[col].astype(str).str.strip()

    # 4️⃣ Intentar volver a convertir a numérico donde sea posible
    df_limpio = df_limpio.apply(pd.to_numeric, errors="ignore")

    return df_limpio

# Cargar el DataFrame sucio que generaste
df_sucio = pd.read_csv("df_sucio.csv")

# Limpiar el DataFrame
df_limpio = limpiar_df(df_sucio)

# Guardar el resultado limpio
df_limpio.to_csv("df_limpio.csv", index=False)

# Verificar los cambios
print("✅ Limpieza completada.")
print("\n--- Vista previa del DataFrame limpio ---")
print(df_limpio.head())
print("\n--- Comprobación de NaNs ---")
print(df_limpio.isnull().sum())


C:\Users\patri\AppData\Local\Temp\ipykernel_15816\2544998163.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_limpio = df_limpio.apply(pd.to_numeric, errors="ignore")


✅ Limpieza completada.

--- Vista previa del DataFrame limpio ---
                    name  release_date  price  positive  negative   app_id  \
0           Train Bandit  Oct 12, 2017   0.99      53.0       5.0   655370   
1               Henosis™  Jul 23, 2020   5.99       3.0       0.0  1355720   
2  Two Weeks in Painland   Feb 3, 2020   0.00      50.0       8.0  1139950   
3         Wartune Reborn  Feb 26, 2021   0.00      87.0      49.0        0   
4              TD Worlds   Jan 9, 2022  10.99      21.0       7.0  1659180   

   min_owners  max_owners  hltb_single  
0           0     20000.0          0.0  
1           0     20000.0          0.0  
2           0         0.0          0.0  
3       50000    100000.0          0.0  
4           0     20000.0          0.0  

--- Comprobación de NaNs ---
name            0
release_date    0
price           0
positive        0
negative        0
app_id          0
min_owners      0
max_owners      0
hltb_single     0
dtype: int64


## **Resultado final**

In [ ]:
df_limpio.isnull().sum()

name            0
release_date    0
price           0
positive        0
negative        0
app_id          0
min_owners      0
max_owners      0
hltb_single     0
dtype: int64

In [36]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77628 entries, 0 to 77627
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          77628 non-null  object 
 1   release_date  77628 non-null  object 
 2   price         77628 non-null  float64
 3   positive      77628 non-null  float64
 4   negative      77628 non-null  float64
 5   app_id        77628 non-null  int64  
 6   min_owners    77628 non-null  int64  
 7   max_owners    77628 non-null  float64
 8   hltb_single   77628 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 5.3+ MB


In [37]:
df_limpio.shape

(77628, 9)

## **Cargar csv limpio y sucio a mi carpeta local**

In [ ]:
import pandas as pd
import numpy as np
import random

# --- Función para ensuciar el DataFrame ---
def ensuciar_df(df_original, columnas_protegidas=None):
    df = df_original.copy()

    # Columnas que no deben ensuciarse
    if columnas_protegidas is None:
        columnas_protegidas = ['name', 'release_date']

    # --- Duplicar algunas filas ---
    num_duplicados = random.randint(1, int(len(df) * 0.3))
    duplicadas = df.sample(n=num_duplicados, replace=True)
    df = pd.concat([df, duplicadas], ignore_index=True)

    # --- Insertar valores 'bbb' en columnas no protegidas ---
    num_invalid = int(len(df) * 0.02)
    columnas_modificables = [c for c in df.columns if c not in columnas_protegidas]

    for col in random.sample(columnas_modificables, len(columnas_modificables) // 2):
        if df[col].dtype != 'object':
            df[col] = df[col].astype('object')
        valid_indices = df.index[df[col].notna()]
        if len(valid_indices) > 0:
            invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)
            df.loc[invalid_indices, col] = 'bbb'

    # --- Cambiar tipos de formato en columnas no protegidas ---
    columnas_a_cambiar = random.sample(columnas_modificables, random.randint(1, len(columnas_modificables) // 2))
    for col in columnas_a_cambiar:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(str)
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- Insertar NaNs en un 5% de las celdas, excepto las protegidas ---
    num_nan = int(len(df) * 0.05)
    for col in columnas_modificables:
        nan_indices = np.random.choice(df.index, size=min(num_nan, len(df)), replace=False)
        df.loc[nan_indices, col] = np.nan

    return df


# --- Cargar el CSV original ---
ruta_csv = 'https://raw.githubusercontent.com/ibrahimelCABALLOesquizofrenico/Proyecto-cienciasdedatos/refs/heads/main/games%20steam%202013-2023.csv?token=GHSAT0AAAAAADMMHO5D7IYJ4HS6BMAFWD5Q2HU7PFQ'
df = pd.read_csv(ruta_csv)

# --- Aplicar el ensuciado ---
df_sucio = ensuciar_df(df, columnas_protegidas=['name', 'release_date'])

# --- Guardar el DataFrame sucio en un nuevo archivo CSV ---
df_sucio.to_csv('df_sucio.csv', index=False, encoding='utf-8-sig')

# --- Mensaje de confirmación ---
print("Base de datos sucia creada exitosamente como 'df_sucio.csv'")


Base de datos sucia creada exitosamente como 'df_sucio.csv'


In [35]:
import pandas as pd
import numpy as np

# --- Función para limpiar el DataFrame ---
def limpiar_df(df):
    df_limpio = df.copy()

    # Reemplazar los valores "bbb" (o cualquier otro texto basura) por NaN
    df_limpio = df_limpio.replace('bbb', np.nan)

    # Reemplazar los NaN con 0
    df_limpio = df_limpio.fillna(0)

    # Asegurar que las columnas numéricas vuelvan a ser tipo numérico cuando sea posible
    for col in df_limpio.columns:
        df_limpio[col] = pd.to_numeric(df_limpio[col], errors='ignore')

    return df_limpio


# --- Cargar el CSV sucio ---
df_sucio = pd.read_csv('df_sucio.csv')

# --- Aplicar la limpieza ---
df_limpio = limpiar_df(df_sucio)

# --- Guardar el DataFrame limpio ---
df_limpio.to_csv('df_limpio.csv', index=False, encoding='utf-8-sig')

# --- Confirmación ---
print(" Base de datos limpia creada exitosamente como 'df_limpio.csv")


C:\Users\patri\AppData\Local\Temp\ipykernel_15816\2316821566.py:16: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_limpio[col] = pd.to_numeric(df_limpio[col], errors='ignore')


 Base de datos limpia creada exitosamente como 'df_limpio.csv


## **Traduccion de columnnas por medio de un diccionario y la funcion remplace**

# Traduccion o que significa cada columna en español

name -> nombre(nombre del videojuego)

release_date -> fecha de salida(cuando salio el juego)

price -> precio(precio del videojuego)

positive -> positivo(calificacion positiva del juego)

negative -> negativo(calificacion negativa del juego)

app_id -> id de la app(numero que representa al juego)

min_owners -> minimo de dueños(cantidad minima de dueños que tiene el juego)

max_owners -> maximo de dueños(cantidad maxima de dueños que tuvo el juego)

hltb_single -> duracion de canpaña en solitario(la duracion en horas mde la campaña en solitario (es aproximado))

In [41]:
import pandas as pd

# --- Diccionario de traducción ---
traducciones = {
    "name": "nombre",
    "release_date": "fecha_de_lanzamiento",
    "price": "precio",
    "positive": "calificacion_positivas",
    "negative": "calificacion_negativas",
    "app_id": "id_del_juego",
    "min_owners": "minimo_de_dueños",
    "max_owners": "maximo_de_dueños",
    "hltb_single": "duracion_de_la_campaña_aprox",
    }

# --- Función para traducir columnas ---
def traducir_columnas(df, diccionario):
    nuevos_nombres = {col: diccionario.get(col, col) for col in df.columns}
    df.rename(columns=nuevos_nombres, inplace=True)
    return df, nuevos_nombres

# --- Traducir el csv sucio ---
try:
    df_sucio = pd.read_csv("df_sucio.csv", encoding="utf-8")
    df_sucio, nombres_sucio = traducir_columnas(df_sucio, traducciones)
    df_sucio.to_csv("df_sucio_es.csv", index=False, encoding="utf-8-sig")
    print(" Archivo 'df_sucio.csv' traducido y guardado como 'df_sucio_es.csv'")
except FileNotFoundError:
    print("No se encontró el archivo 'df_sucio.csv'. Asegúrate de tenerlo en la misma carpeta.")

# --- Traducir el csv limpio ---
try:
    df_limpio = pd.read_csv("df_limpio.csv", encoding="utf-8")
    df_limpio, nombres_limpio = traducir_columnas(df_limpio, traducciones)
    df_limpio.to_csv("df_limpio_es.csv", index=False, encoding="utf-8-sig")
    print(" Archivo 'df_limpio.csv' traducido y guardado como 'df_limpio_es.csv'")
except FileNotFoundError:
    print(" No se encontró el archivo 'df_limpio.csv'. Asegúrate de tenerlo en la misma carpeta.")

# --- Mostrar resumen de traducciones ---
print("\nTraducciones aplicadas:")
for clave, valor in traducciones.items():
    print(f"{clave} → {valor}")


 Archivo 'df_sucio.csv' traducido y guardado como 'df_sucio_es.csv'
 Archivo 'df_limpio.csv' traducido y guardado como 'df_limpio_es.csv'

Traducciones aplicadas:
name → nombre
release_date → fecha_de_lanzamiento
price → precio
positive → calificacion_positivas
negative → calificacion_negativas
app_id → id_del_juego
min_owners → minimo_de_dueños
max_owners → maximo_de_dueños
hltb_single → duracion_de_la_campaña_aprox
